In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import nltk
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import random
import copy

# preprocess documents - lemmatization and stemming

def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

def process_training_data():
    """Fetches training data from sklearn and preprocesses them for further modelling.

    Returns
    -------
    doc_cnt
        a number of docs in training data
    wrd_cnt
        a number of words in dictionary
    docs
        a list of preprocessed docs
    dictionary
        dictionary extracted from training data
    maxdoclen
        a length of the longest doc
    """
    newsgroups_train = fetch_20newsgroups(subset='train')
    print(len(newsgroups_train.data), " documents loaded.")

    print("Example document:")
    print(newsgroups_train.data[0])

    processed_docs = list(map(preprocess, newsgroups_train.data))

    print("Example document - lemmatized and stemmed:")
    print(processed_docs[0])


    # Construct dictionary

    dictionary = gensim.corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

    print("Dictionary size: ", len(dictionary))

    # Filter words in documents

    docs = list()
    maxdoclen = 0 
    for doc in processed_docs:
        docs.append(list(filter(lambda x: x != -1, dictionary.doc2idx(doc))))
        maxdoclen = max(maxdoclen, len(docs[-1]))

    print("Example document - filtered:")
    print(docs[0])

    print("Maximum document length:", maxdoclen)

    doc_cnt = len(docs)
    wrd_cnt = len(dictionary)
    
    return doc_cnt, wrd_cnt, docs, dictionary, maxdoclen

def process_test_data(dictionary):
    """Fetches test data from sklearn and preprocesses them to test the model.

    Parameters
    ----------
    dictionary : str
        Dictionary taken from training data

    Returns
    -------
    doc_cnt_test
        a number of docs in test data
    docs_test
        a list of preprocessed docs
    """
    newsgroups_test = fetch_20newsgroups(subset='test')
    print(len(newsgroups_test.data), " documents loaded.")

    print("Example document:")
    print(newsgroups_test.data[0])


    # preprocess documents - lemmatization and stemming

    processed_docs_test = list(map(preprocess, newsgroups_test.data))

    print("Example document - lemmatized and stemmed:")
    print(processed_docs_test[0])


    # filter words in documents

    docs_test = list()
    maxdoclen_test = 0 
    for doc in processed_docs_test:
        docs_test.append(list(filter(lambda x: x != -1, dictionary.doc2idx(doc))))
        maxdoclen_test = max(maxdoclen_test, len(docs_test[-1]))

    print("Example document - filtered:")
    print(docs_test[0])

    print("Maximum document length:", maxdoclen_test)


    doc_cnt_test = len(docs_test)
    
    return doc_cnt_test, docs_test

def entropy_per_topic(gamma, cw, ck, wrd_cnt, topics):
    """Computes entropy per topic

    Parameters
    ----------
    gamma : float
        Hyperparameter for Gibbs sampling
    cw : list
        How many times every word is assigned to topic k
    ck : list
        How many words are assigned to topic k
    wrd_cnt : int
        A number of words in dictionary
    topics : int
        A number of topics
    Returns
    -------
    H_k
        a list of entropies for each topic
    """
    H_k = []
    for k in range(topics):
        probs_of_words = (cw[:,k] + gamma)/(wrd_cnt*gamma + ck[k])
        H_k.append(-np.sum(np.multiply(probs_of_words, np.log2(probs_of_words))))
    return H_k

# set the hyperparameters for Latent Dirichlet Allocation
iterations = 100 
topics = 20 
alpha = 0.01 # hyperparameter for Gibbs sampling
gamma = 0.01 # hyperparameter for Gibbs sampling

# preprocess training and test data
doc_cnt, wrd_cnt, docs, dictionary, maxdoclen = process_training_data()

doc_cnt_test, docs_test = process_test_data(dictionary)

# initialize the random seed
random.seed(13)

# find the longest doc in training data
longest_doc = 0
length = 0
i = 0
for doc in docs:
    if len(doc) > length:
        length = len(doc)
        longest_doc = i
    i += 1

topics_in_longest_doc = [[0 for x in range(maxdoclen)] for y in range(8)]

# initialization for Gibbs sampling
z_nd = list()
temp = []
for doc in docs:
    for i in range(len(doc)):
        temp.append(random.randint(0,19))
    z_nd.append(temp)
    temp = []

c_d = [[0 for x in range(topics)] for y in range(doc_cnt)] # how many words in doc d are assigned to topic k
c_w = [[0 for x in range(topics)] for y in range(wrd_cnt)] # how many times the word is assigned to topic k
c_k = [0 for x in range(topics)] # how many words assigned to topic k

i = 0
for doc in z_nd:
    for word in doc:
        c_d[i][word] += 1
    i += 1
    
for doc_w, doc_k in zip(docs, z_nd):
    for word, k in zip(doc_w, doc_k):
        c_w[word][k] += 1
        
for doc in z_nd:
    for word in doc:
        c_k[word] += 1

# initial distribution over topics in the longest document
topics_in_longest_doc[0] = copy.deepcopy(z_nd[longest_doc])

save_distr = [1, 2, 5, 10, 20, 50]
num = 1
cw = np.asarray(c_w)
cd = np.asarray(c_d)
ck = np.asarray(c_k)

# main loop
K = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
for i in range(iterations):
    if i in save_distr:
# save distribution over topics in the longest document after 1st, 2nd, 5th, 10th, 20th, 50th iterations
        topics_in_longest_doc[num] = copy.deepcopy(z_nd[longest_doc])
        num += 1
# start compute word entropy for each topic after 1st, 2nd, 5th, 10th, 20th, 50th iteration
        H_k = entropy_per_topic(gamma, cw, ck, wrd_cnt, topics)
        print(f'entropy after {i}th iteration')
        print(H_k)        
# end word entropy computation
    for d in range(doc_cnt):
        for n in range(len(docs[d])):
            z = z_nd[d][n]
            cd[d,z] -= 1
            w_nd = docs[d][n]
            cw[w_nd,z] -= 1
            ck[z] -= 1
            pr = (alpha + cd[d,:])/(topics*alpha + len(docs[d]) - 1) * (gamma + cw[w_nd,:])/(wrd_cnt*gamma + ck)
            sum_pr = np.sum(pr)
            pr = pr/sum_pr
            p = np.ndarray.tolist(pr)
            value = random.choices(K, weights=p, k=1)
            topic = value
            z_nd[d][n] = topic
            cd[d, topic] += 1
            cw[w_nd, topic] += 1
            ck[topic] += 1
            
#Perplexity of the model
H_k = entropy_per_topic(gamma, cw, ck, wrd_cnt, topics)
print(f'entropy after 100th iteration')
print(H_k) 

entropy = np.asarray(H_k)
Perp = 2**entropy
print('Model perplexity')
print(Perp)

#save model
np.savetxt('model/c_w.csv', cw, delimiter=',')
np.savetxt('model/c_k.csv', ck, delimiter=',')

In [ ]:
# print distribution over topics for the longest document after iterations 0,1,2,5,10,20,50,100
topics_in_longest_doc[num] = z_nd[longest_doc]
distribution = [[0 for x in range(topics)] for y in range(8)]

i = 0
iter = topics_in_longest_doc[0]
for word in iter:
        distribution[i][word] += 1
i += 1
for iter in range(1,len(topics_in_longest_doc)):
    for word in topics_in_longest_doc[iter]:
        w = word[0]
        distribution[i][w] += 1
    i += 1

num_of_iter = [0, 1, 2, 5, 10, 20, 50, 100]
i = 0
for iteration in distribution:
    print(f'{num_of_iter[i]} iteration')
    print(iteration)
    i += 1

In [ ]:
def most_frequent_words(topic, words):
    """Prints the most frequent words for a given topic

    Parameters
    ----------
    topic : int
        Topic
    words : list
        All words that sre assigned to a given topic
    """
    dict_word_freq = {}
    for word in words:
        if word in dict_word_freq.keys():
            dict_word_freq[word] += 1
        else:
            dict_word_freq[word] = 1
    sorted_dict = dict(sorted(dict_word_freq.items(), key=lambda item: item[1], reverse = True))
    d_items = sorted_dict.items()
    i = 0
    print(f'Words for topic {topic}')
    for key, value in d_items:
        print(f'Word {key} appears {value} times in topic {topic}')
        i += 1
        if i == 20:
            break
        
# print most frequent words for 3 topics
words_2 = []
words_10 = []
words_16 = []

idx_to_words = {v: k for k, v in dictionary.token2id.items()}

znd = []
for doc in z_nd:
    doc_z = []
    for word in doc:
        doc_z.append(word[0])
    znd.append(doc_z)

# create lists of all words that sre assigned to topics 2, 10, 16
for doc_z, doc_w in zip(znd, docs):
    for i in range(len(doc_z)):
        if doc_z[i] == 2:
            words_2.append(idx_to_words[doc_w[i]])
        else:
            if doc_z[i] == 10:
                words_10.append(idx_to_words[doc_w[i]])
            else:
                if doc_z[i] == 16:
                    k = doc_w[i]
                    words_16.append(idx_to_words[doc_w[i]])

most_frequent_words(2, words_2)
most_frequent_words(10, words_10)
most_frequent_words(16, words_16)

In [ ]:
# Model testing 
def entropy_test(gamma, cw, cd, ck, docs, wrd_cnt, topics):
    """Computes entropy f test data

    Parameters
    ----------
    gamma : float
        Hyperparameter for Gibbs sampling
    cw : list
        How many times every word is assigned to topic k
    cd : list
        How many words in doc d are assigned to topic k
    ck : list
        How many words are assigned to topic k
    docs : list
        List of all docs in test data
    wrd_cnt : int
        A number of words in dictionary
    topics : int
        A number of topics
    Returns
    -------
    H
        Entropy for test data
    """
    probs_of_words = []
    N = 0
    for i in range(len(docs)):
        for word in docs[i]:
            p_k = (gamma + cw[word,:])/(wrd_cnt*gamma + ck) * (alpha + cd[i,:])/(topics*alpha + len(docs[i]))
            probs_of_words.append(np.sum(p_k))
        N += len(docs[i])
    p_log = np.log2(probs_of_words)
    
    sum_log = 0
    for doc in docs:
        for word in doc:
            sum_log += p_log[word]
        
    H = - 1/N * sum_log
    return H

#initialization
z_nd_test = list()
temp = []
for doc in docs_test:
    for i in range(len(doc)):
        temp.append(random.randint(0,19))
    z_nd_test.append(temp)
    temp = []

c_d_test = [[0 for x in range(topics)] for y in range(doc_cnt_test)] #how many words in doc d are assigned to topic k
c_w = np.loadtxt('model/cw.csv', delimiter=',')
c_k = np.loadtxt('model/ck.csv')

i = 0
for doc in z_nd_test:
    for word in doc:
        c_d_test[i][word] += 1
    i += 1

cd_test = np.asarray(c_d_test)

# main loop
K = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
for i in range(50):
    for d in range(doc_cnt_test):
        for n in range(len(docs_test[d])):
            z = z_nd_test[d][n]
            cd_test[d,z] -= 1
            pr = (alpha + cd_test[d,:])/(topics*alpha + len(docs_test[d]) - 1) * (gamma + cw[w_nd,:])/(wrd_cnt*gamma + ck)
            sum_pr = np.sum(pr)
            pr = pr/sum_pr
            p = np.ndarray.tolist(pr)
            value = random.choices(K, weights=p, k=1)
            topic = value
            z_nd_test[d][n] = topic
            cd_test[d, topic] += 1 

#Perplexity of the model
H_test = entropy_test(gamma, c_w, cd_test, c_k, docs_test, wrd_cnt, topics)
print(f'test entropy after 50th iteration')
print(H_test) 

PP_test = 2**H_test
print('Test model perplexity')
print(PP_test)

In [ ]:
# Simple bayesian model using only one distribution over words for all documents with symmetric Dirichlet prior (to compare with LDA)

c_w = np.loadtxt('model/cw.csv', delimiter=',')

N = 0
for doc_d in docs_test:
    N += len(doc_d)
    
gamma_simple = 0.1
cw_w = np.sum(c_w, axis=1)
pw = (gamma_simple + cw_w)/(wrd_cnt*gamma_simple + np.sum(cw_w))
log_pw = np.log2(pw)


sum_log = 0
for doc in docs_test:
    for word in doc:
        sum_log += log_pw[word]

H_simple = - 1/N * sum_log

PP_simple = 2**H_simple
print('Simple bayesian model perplexity')
print(PP_simple)